In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import random
import shutil
import plotly
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')


# Initial Features

These initial features include:
- Rolling season averages for each of the team metrics per game (around 60 metrics)
- The difference between the two team's rolling averages.  


### Analyze the Boxscore Dataframe that we will be using to solve 

In [4]:
df = pd.read_csv('data/team/aggregates/Both_Team_Boxscores_ALL_with_Trad_Difs.csv')
df.head()

,Unnamed: 0,trad_unnamed: 0,trad_team,trad_matchup,trad_gamedate,trad_w/l,trad_min,trad_pts,trad_fgm,trad_fga,...,t1_t2_ft_percent,t1_t2_oreb,t1_t2_dreb,t1_t2_reb,t1_t2_ast,t1_t2_stl,t1_t2_blk,t1_t2_tov,t1_t2_pf,who_wins
0,1513,1514,LAL,LAL vs. SAC,2010-01-01,W,48.0,109.0,39.0,86.0,...,0.043,11.0,-2.0,9.0,-11.0,-2.0,5.0,-2.0,-9.0,t1
1,1512,1513,ATL,ATL vs. NYK,2010-01-01,L,53.0,108.0,46.0,100.0,...,-0.004,7.0,-5.0,2.0,1.0,2.0,5.0,-6.0,2.0,t2
2,1511,1512,ORL,ORL @ MIN,2010-01-01,W,48.0,106.0,36.0,80.0,...,0.076,-8.0,-1.0,-9.0,4.0,9.0,1.0,-7.0,-5.0,t1
3,1510,1511,NYK,NYK @ ATL,2010-01-01,W,53.0,112.0,45.0,88.0,...,0.004,-7.0,5.0,-2.0,-1.0,-2.0,-5.0,6.0,-2.0,t1
4,1509,1510,MIN,MIN vs. ORL,2010-01-01,L,48.0,94.0,37.0,86.0,...,-0.076,8.0,1.0,9.0,-4.0,-9.0,-1.0,7.0,5.0,t2


In [3]:
colz= list(df.columns)

In [4]:
df.trad_season.unique()

array([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2022], dtype=int64)

In [5]:
df['my_gameid'] = df['trad_matchup'] + '_' + df['trad_gamedate'].replace('/','_')

### Make trad_gamedates work

In [6]:
# fix trad_gamedates
df['trad_gamedate'] = pd.to_datetime(df['trad_gamedate'])
df['trad_gamedate'] = df['trad_gamedate'].dt.strftime('%Y-%m-%d')
df.sort_values('trad_gamedate', ascending = False, inplace=True)
df

,Unnamed: 0,trad_unnamed: 0,trad_team,trad_matchup,trad_gamedate,trad_w/l,trad_min,trad_pts,trad_fgm,trad_fga,...,t1_t2_oreb,t1_t2_dreb,t1_t2_reb,t1_t2_ast,t1_t2_stl,t1_t2_blk,t1_t2_tov,t1_t2_pf,who_wins,my_gameid
33065,29928,19,CHI,CHI @ NYK,2022-12-23,W,48.0,118.0,45.0,93.0,...,-1.0,-5.0,-6.0,1.0,4.0,0.0,-6.0,5.0,t1,CHI @ NYK_2022-12-23
33050,29926,17,NYK,NYK vs. CHI,2022-12-23,L,48.0,117.0,43.0,86.0,...,1.0,5.0,6.0,-1.0,-4.0,0.0,6.0,-5.0,t2,NYK vs. CHI_2022-12-23
33038,29924,15,TOR,TOR @ CLE,2022-12-23,W,48.0,118.0,42.0,84.0,...,5.0,5.0,10.0,-2.0,1.0,-2.0,4.0,3.0,t1,TOR @ CLE_2022-12-23
33039,29937,28,PHI,PHI vs. LAC,2022-12-23,W,48.0,119.0,47.0,88.0,...,0.0,6.0,6.0,6.0,2.0,-6.0,-2.0,4.0,t1,PHI vs. LAC_2022-12-23
33040,29936,27,MIA,MIA vs. IND,2022-12-23,L,48.0,108.0,41.0,95.0,...,4.0,-9.0,-5.0,-3.0,2.0,-3.0,-6.0,-10.0,t2,MIA vs. IND_2022-12-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,1511,1512,ORL,ORL @ MIN,2010-01-01,W,48.0,106.0,36.0,80.0,...,-8.0,-1.0,-9.0,4.0,9.0,1.0,-7.0,-5.0,t1,ORL @ MIN_2010-01-01
1,1512,1513,ATL,ATL vs. NYK,2010-01-01,L,53.0,108.0,46.0,100.0,...,7.0,-5.0,2.0,1.0,2.0,5.0,-6.0,2.0,t2,ATL vs. NYK_2010-01-01
4,1509,1510,MIN,MIN vs. ORL,2010-01-01,L,48.0,94.0,37.0,86.0,...,8.0,1.0,9.0,-4.0,-9.0,-1.0,7.0,5.0,t2,MIN vs. ORL_2010-01-01
5,1508,1509,SAC,SAC @ LAL,2010-01-01,L,48.0,108.0,43.0,79.0,...,-11.0,2.0,-9.0,11.0,2.0,-5.0,2.0,9.0,t2,SAC @ LAL_2010-01-01


### From here, df is 2016 on

In [7]:
df = df[df.trad_season >= 2016]
df

,Unnamed: 0,trad_unnamed: 0,trad_team,trad_matchup,trad_gamedate,trad_w/l,trad_min,trad_pts,trad_fgm,trad_fga,...,t1_t2_oreb,t1_t2_dreb,t1_t2_reb,t1_t2_ast,t1_t2_stl,t1_t2_blk,t1_t2_tov,t1_t2_pf,who_wins,my_gameid
33065,29928,19,CHI,CHI @ NYK,2022-12-23,W,48.0,118.0,45.0,93.0,...,-1.0,-5.0,-6.0,1.0,4.0,0.0,-6.0,5.0,t1,CHI @ NYK_2022-12-23
33050,29926,17,NYK,NYK vs. CHI,2022-12-23,L,48.0,117.0,43.0,86.0,...,1.0,5.0,6.0,-1.0,-4.0,0.0,6.0,-5.0,t2,NYK vs. CHI_2022-12-23
33038,29924,15,TOR,TOR @ CLE,2022-12-23,W,48.0,118.0,42.0,84.0,...,5.0,5.0,10.0,-2.0,1.0,-2.0,4.0,3.0,t1,TOR @ CLE_2022-12-23
33039,29937,28,PHI,PHI vs. LAC,2022-12-23,W,48.0,119.0,47.0,88.0,...,0.0,6.0,6.0,6.0,2.0,-6.0,-2.0,4.0,t1,PHI vs. LAC_2022-12-23
33040,29936,27,MIA,MIA vs. IND,2022-12-23,L,48.0,108.0,41.0,95.0,...,4.0,-9.0,-5.0,-3.0,2.0,-3.0,-6.0,-10.0,t2,MIA vs. IND_2022-12-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16968,18251,2460,NYK,NYK @ CLE,2016-10-25,L,48.0,88.0,32.0,87.0,...,2.0,-11.0,-9.0,-14.0,-6.0,1.0,3.0,0.0,t2,NYK @ CLE_2016-10-25
16969,18250,2459,GSW,GSW vs. SAS,2016-10-25,L,48.0,100.0,40.0,85.0,...,-13.0,-7.0,-20.0,-1.0,-2.0,3.0,2.0,0.0,t2,GSW vs. SAS_2016-10-25
16973,18246,2455,UTA,UTA @ POR,2016-10-25,L,48.0,104.0,40.0,82.0,...,1.0,-4.0,-3.0,-3.0,4.0,2.0,1.0,1.0,t2,UTA @ POR_2016-10-25
16971,18248,2457,CLE,CLE vs. NYK,2016-10-25,W,48.0,117.0,45.0,94.0,...,-2.0,11.0,9.0,14.0,6.0,-1.0,-3.0,0.0,t1,CLE vs. NYK_2016-10-25


In [8]:
df_reg = df[df.trad_season_type == 'Regular']

In [9]:
#team_games_2016
games = df['my_gameid'].unique()
games

array(['CHI @ NYK_2022-12-23', 'NYK vs. CHI_2022-12-23',
       'TOR @ CLE_2022-12-23', ..., 'UTA @ POR_2016-10-25',
       'CLE vs. NYK_2016-10-25', 'POR vs. UTA_2016-10-25'], dtype=object)

In [10]:
def get_avgs_by_game(my_gameid, team, season, game_date, metric):
    data = df[df['trad_team'] == team]
    data = data[data['trad_season'] == season]
    data = data[data['trad_season_type'] == 'Regular']
    # filter by date
    data = data[data['trad_gamedate'] < game_date]
    data = data.dropna(subset = [metric])
    metric = data[metric].mean()
    return metric

In [11]:
# test on a random row from df
random_row = df.sample(1)
get_avgs_by_game(random_row['my_gameid'].values[0], random_row['trad_team'].values[0], random_row['trad_season'].values[0], random_row['trad_gamedate'].values[0], 'trad_pts')

97.83333333333333

### Add Team 1 running averages

Question: What are the most important features to keep? Should we just make as many as possible and analyze from there? It does take a bit of processing power to calculate these metrics. 

In [12]:
colz = pd.DataFrame(df.columns)
colz

,0
0,Unnamed: 0
1,trad_unnamed: 0
2,trad_team
3,trad_matchup
4,trad_gamedate
...,...
226,t1_t2_blk
227,t1_t2_tov
228,t1_t2_pf
229,who_wins


In [13]:
#get possible features from non-object columns
# identify columns that are numeric
numeric_cols = [col for col in df.columns if df[col].dtype in ['int64', 'float64']]
numeric_cols

['Unnamed: 0',
 'trad_unnamed: 0',
 'trad_min',
 'trad_pts',
 'trad_fgm',
 'trad_fga',
 'trad_fg%',
 'trad_3pm',
 'trad_3pa',
 'trad_3p%',
 'trad_ftm',
 'trad_fta',
 'trad_ft%',
 'trad_oreb',
 'trad_dreb',
 'trad_reb',
 'trad_ast',
 'trad_tov',
 'trad_stl',
 'trad_blk',
 'trad_pf',
 'trad_+/-',
 'trad_season',
 'adv_unnamed: 0',
 'adv_min',
 'adv_offrtg',
 'adv_defrtg',
 'adv_netrtg',
 'adv_ast%',
 'adv_ast/to',
 'adv_astratio',
 'adv_oreb%',
 'adv_dreb%',
 'adv_reb%',
 'adv_tov%',
 'adv_efg%',
 'adv_ts%',
 'adv_pace',
 'adv_pie',
 'adv_season',
 'four_unnamed: 0',
 'four_min',
 'four_efg%',
 'four_ftarate',
 'four_tov%',
 'four_oreb%',
 'four_oppefg%',
 'four_oppfta\xa0rate',
 'four_opptov%',
 'four_opporeb%',
 'four_season',
 'misc_unnamed: 0',
 'misc_min',
 'misc_ptsoff\xa0to',
 'misc_2ndpts',
 'misc_fbps',
 'misc_pitp',
 'misc_opp\xa0ptsoff\xa0to',
 'misc_opp2nd\xa0pts',
 'misc_oppfbps',
 'misc_opppitp',
 'misc_season',
 'score_unnamed: 0',
 'score_min',
 'score_%fga2pt',
 'score_%

In [14]:
for col in numeric_cols:
    if 'season' in col:
        numeric_cols.remove(col)

In [15]:
for col in numeric_cols:
    if 'season' in col:
        numeric_cols.remove(col)

In [16]:
features = numeric_cols[2:67]
features

['trad_min',
 'trad_pts',
 'trad_fgm',
 'trad_fga',
 'trad_fg%',
 'trad_3pm',
 'trad_3pa',
 'trad_3p%',
 'trad_ftm',
 'trad_fta',
 'trad_ft%',
 'trad_oreb',
 'trad_dreb',
 'trad_reb',
 'trad_ast',
 'trad_tov',
 'trad_stl',
 'trad_blk',
 'trad_pf',
 'trad_+/-',
 'adv_unnamed: 0',
 'adv_min',
 'adv_offrtg',
 'adv_defrtg',
 'adv_netrtg',
 'adv_ast%',
 'adv_ast/to',
 'adv_astratio',
 'adv_oreb%',
 'adv_dreb%',
 'adv_reb%',
 'adv_tov%',
 'adv_efg%',
 'adv_ts%',
 'adv_pace',
 'adv_pie',
 'four_unnamed: 0',
 'four_min',
 'four_efg%',
 'four_ftarate',
 'four_tov%',
 'four_oreb%',
 'four_oppefg%',
 'four_oppfta\xa0rate',
 'four_opptov%',
 'four_opporeb%',
 'misc_unnamed: 0',
 'misc_min',
 'misc_ptsoff\xa0to',
 'misc_2ndpts',
 'misc_fbps',
 'misc_pitp',
 'misc_opp\xa0ptsoff\xa0to',
 'misc_opp2nd\xa0pts',
 'misc_oppfbps',
 'misc_opppitp',
 'score_unnamed: 0',
 'score_min',
 'score_%fga2pt',
 'score_%fga3pt',
 'score_%pts2pt',
 'score_%pts2pt\xa0mr',
 'score_%pts3pt',
 'score_%ptsfbps',
 'score_%p

### Create average features for each of these for each team

In [17]:
# ignore warnings
warnings.filterwarnings('ignore')

In [18]:
# Note: 72 min runtime

for f in features:
    df['t1_running_' + f] = df.apply(lambda row: get_avgs_by_game(row['my_gameid'], row['trad_team'], row['trad_season'], row['trad_gamedate'], f), axis=1)

#### Add Team 2

In [19]:
# Note: 66 min runtime

for f in features:
    df['t2_running_' + f] = df.apply(lambda row: get_avgs_by_game(
                                                row['my_gameid'], 
                                                row['team_2'], 
                                                row['trad_season'], 
                                                row['trad_gamedate'], 
                                                f), 
                                                axis=1)

### Add dif between teams

In [20]:
for f in features:
    df['running_t1-t2_' + f] = df['t1_running_' + f] - df['t2_running_' + f]

In [21]:
df.to_csv('data/team/aggregates/Boxes_GameDifferences_and_RunningSeasonAverages.csv')

### Load from here on pickup

In [22]:
df = pd.read_csv('data/team/aggregates/Boxes_GameDifferences_and_RunningSeasonAverages.csv')

### Add All Teams Running (Mean of all Teams) avgs (to compare)

For this we need to change the function to all teams

In [23]:
def get_league_avgs_by_game(my_gameid, season, game_date, metric):
    data = df[df['trad_season'] == season]
    data = data[data['trad_season_type'] == 'Regular']
    # filter by date
    data = data[data['trad_gamedate'] < game_date]
    data = data.dropna(subset = [metric])
    metric = data[metric].mean()
    return metric

In [24]:
# Takes 140 minutes to run

for f in features:
    df['league_running_' + f] = df.apply(lambda row: get_league_avgs_by_game(row['my_gameid'], row['trad_season'], row['trad_gamedate'], f), axis=1)

In [25]:
df.to_csv('data/team/aggregates/Boxes_GameDifferences_and_RunningSeasonAverages_and_T1-T2_Running_Averages_and_League_Running_Averages.csv')

### Add Both Teams Delta from the Mean  of All Teams
For each metric

In [26]:
df = pd.read_csv('data/team/aggregates/Boxes_GameDifferences_and_RunningSeasonAverages_and_T1-T2_Running_Averages_and_League_Running_Averages.csv')

In [27]:
colz = df.columns
colz = [c for c in colz if 'league_running' in c]
list(colz)

['league_running_trad_min',
 'league_running_trad_pts',
 'league_running_trad_fgm',
 'league_running_trad_fga',
 'league_running_trad_fg%',
 'league_running_trad_3pm',
 'league_running_trad_3pa',
 'league_running_trad_3p%',
 'league_running_trad_ftm',
 'league_running_trad_fta',
 'league_running_trad_ft%',
 'league_running_trad_oreb',
 'league_running_trad_dreb',
 'league_running_trad_reb',
 'league_running_trad_ast',
 'league_running_trad_tov',
 'league_running_trad_stl',
 'league_running_trad_blk',
 'league_running_trad_pf',
 'league_running_trad_+/-',
 'league_running_adv_unnamed: 0',
 'league_running_adv_min',
 'league_running_adv_offrtg',
 'league_running_adv_defrtg',
 'league_running_adv_netrtg',
 'league_running_adv_ast%',
 'league_running_adv_ast/to',
 'league_running_adv_astratio',
 'league_running_adv_oreb%',
 'league_running_adv_dreb%',
 'league_running_adv_reb%',
 'league_running_adv_tov%',
 'league_running_adv_efg%',
 'league_running_adv_ts%',
 'league_running_adv_pace',
 

In [28]:
features = features[:-1]

#### I want to add:
- t1_running_f - league_running_f, 
- T2-avg, and 
- (T1-avg) - (T2-avg)

In [29]:
#t1_running - league_running
for f in features:
    df['t1_league_delta_' + f] = df['t1_running_' + f] - df['league_running_' + f]


In [30]:
for f in features:
    df['t2_league_delta_' + f] = df['t2_running_' + f] - df['league_running_' + f]

In [31]:
for f in features:
    df['t1_delta_minus_t2_delta_' + f] = df['t1_league_delta_' + f] - df['t2_league_delta_' + f]

In [32]:
df.to_csv('data/team/aggregates/Boxes_GameDifferences_and_RunningSeasonAverages_and_T1-T2_Running_Averages_and_League_Running_Averages_and_T1-T2_League_Delta.csv')